In [29]:
import pandas as pd
import numpy as np

# Очистка данных

In [30]:
data = pd.read_csv('anime.csv')
print(data.shape)
row, col = data.shape

(64, 11)


In [31]:
#data.head()

### Необходимые изменения
**Каких данных не хватает?**
 - Студия ✔
 - Год релиза ✔
 - Сезон релиза(время года) ✔
 - Номер сезона/части ✔


**Какие данные лишние?**
 - "смотреть онлайн" в названиях ✔
 - заголовки перед описаниями ✔

**Какие данные нуждаются в преобразованиях?**
 - Альтернативные названия сделать списком ✔
 - Жанры удобнее хранить в числовом виде (лучше заменить при записи)

In [32]:
# удаляю "смотреть онлайн" после названий
data['Rus_name'] = data['Rus_name'].str.split("смотреть онлайн", n = 1, expand = True)[0]
data['Eng_name'] = data['Eng_name'].str.split("смотреть онлайн", n = 1, expand = True)[0]

f_c = len("Описание аниме")
s_c = len("Альтернативное описание аниме")
# удаляю заголовки перед описаниями
data['Description'] = data['Description'].str[f_c:]
data['Alt_description'] = data['Alt_description'].str[s_c:]

# Альтернативные названия преобразовываются в список
data['Minor_names'] = data['Minor_names'].str.split('/').tolist()

In [33]:
data.head(1)

,Page,Rus_name,Eng_name,Href,Genres,Rating,Description,Alt_description,Img,Minor_names,Additional
0,1,Стальной алхимик: Братство,Hagane no Renkinjutsushi (2009) / FMA (2009),/catalog/hagane-no-renkinjutsushi-2009-2400,"['Экшен', 'Военное', 'Приключения', 'Комедия',...",9.2,Ремейк одноимённого аниме-сериала «Стальной ал...,Нарушив главный запрет Алхимии и попытавшись в...,/posters/2400.10127424713.jpg,"[Стальной алхимик: Братство , Hagane no Renki...",На этой странице вы можете посмотреть аниме Ha...


In [34]:
data['Additional'][0]

'На этой странице вы можете посмотреть аниме Hagane no Renkinjutsushi (2009) онлайн и в хорошем качестве. Для начала просмотра выберите серию (см. выше). После выбора серии можно выбрать перевод (русская озвучка или субтитры). Если вам понравилось аниме Стальной алхимик: Братство, то советуем обратить внимание на жанры экшен, военное, приключения, а также комедия, драма, магия, фэнтези, сёнен. Ещё вы можете посмотреть другие аниме от студии Bones или другие аниме в весеннем сезоне 2009 года.'

In [35]:
data['Additional'] = data['Additional'].str.split('.').tolist()
data['Studio'], data['Season'],data['Year'],data['Season_name'],data['Seria_name']=(np.nan for _ in range(5))

In [36]:
for i in range(0,row):
    last_sentence = data['Additional'][i][-2]

    last_sentence = last_sentence.replace(" Ещё вы можете посмотреть другие аниме от студии ","")
    last_sentence = last_sentence.replace(" или другие аниме в ",";")
    last_sentence = last_sentence.replace(" сезоне ",";")
    last_sentence = last_sentence.replace(" года","")
    last_sentence = last_sentence.split(';')

    #проверить что ; работает правильно
    data['Studio'][i] = last_sentence[0]
    data['Season'][i] = last_sentence[1]
    data['Year'][i] = last_sentence[2]
    
    #информация о сезоне
    rus_name = data['Rus_name'][i] 
    rus_name = rus_name.replace(':',' : ')
    rus_name = rus_name.split(' ')
    
    season_name = np.NaN
    result_name = data['Rus_name'][i]
    
    
    if 'сезон' in rus_name:
        season_name = rus_name[rus_name.index('сезон')-1]
        result_name = result_name[:-(7+len(season_name))]  
    elif ':' in rus_name:
        splitted_name = result_name.split(':')
        result_name= splitted_name[0]
        season_name = splitted_name[1]
        
    data['Seria_name'][i] = result_name
    data['Season_name'][i] = season_name

<ipython-input-36-cdeef7b15cba>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Studio'][i] = last_sentence[0]
/home/nastya/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-36-cdeef7b15cba>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Season'][i] = last_sentence[1]
<ipython-input-36-cdeef7b15cba>:

In [37]:
#удаляем столбец тк. он больше не несет полезной информации
del data['Additional']

In [38]:
data.to_csv('cleaned_anime.csv', index=False, encoding='utf-8')

In [39]:
data = pd.read_csv('cleaned_anime.csv')

In [40]:
data.Year[5]

1989.0

# Визуализация

**Что было бы интересно узнать?**
 - Распределение по жанрам(сколько всего записей для каждого жанра существует)
 - Серии с наибольшим числом сезонов(в каких сериях больше всего сезонов(можно считать от 2х)
 - Какой временной промежуток представлен, временной ряд с числом аниме (с учетом сезонов)
 - Распределение по рейтингам (в каком промежутке рейтинга находится большее число аниме)
 - Распределение по студиям (отношение студии и числа выпущенных аниме)
